In [ ]:
!pip install mediapipe

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
#import csv

import json

In [ ]:
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

In [ ]:
def calculate_angle(landmark1, landmark2):
    dy = landmark2[1] - landmark1[1]
    dx = landmark2[0] - landmark1[0]
    angle_radians = np.arctan2(dy, dx)
    angle_degrees = np.degrees(angle_radians)
    return angle_degrees

In [ ]:
def extract_coordinates(results):
    landmarks = results.pose_landmarks.landmark
    coordinates = {
        "left_ear": (landmarks[mp_pose.PoseLandmark.LEFT_EAR.value].x, landmarks[mp_pose.PoseLandmark.LEFT_EAR.value].y),
        "left_shoulder": (landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y)
    }
    return coordinates

In [ ]:
def process_video(video_path, output_path):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error: Cannot open video.")
        return

    rows = []
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(image_rgb)

        if results.pose_landmarks:
            coordinates = extract_coordinates(results)

            # 귀, 어깨 거리 계산
            acromion_distance = abs(coordinates['left_ear'][0] - coordinates['left_shoulder'][0])
            angle = calculate_angle(coordinates['left_ear'], coordinates['left_shoulder'])

            # 거북이 판별
            turtleneck_status = "거북이" if acromion_distance >= 2.5 or angle >= 50 else "정상"
            coordinates['turtleneck_status'] = turtleneck_status
            rows.append(coordinates)

            coordinates['turtleneck_status'] = turtleneck_status
            rows.append(coordinates)

    cap.release()
    save_to_json(output_path, rows)

In [ ]:
def extract_coordinates(results):
    landmarks = results.pose_landmarks.landmark

# left 기준
    coordinates = {
        "left_ear": (landmarks[mp_pose.PoseLandmark.LEFT_EAR.value].x, landmarks[mp_pose.PoseLandmark.LEFT_EAR.value].y),
        "left_shoulder": (landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y)
    }

    return coordinates

In [ ]:
def save_to_json(output_path, rows):
    with open(output_path, 'w') as file:
        json.dump(rows, file, indent=4)

In [ ]:
if __name__ == "__main__":
    video_path = "/content/drive/MyDrive/GDSC_AI/SC_test/test_video/turtle_neck_test_video.mp4"
    output_path = "turtle_neck_results.json"
    process_video(video_path, output_path)